In [84]:
# Para el tratamiento de datos:
import pandas as pd
import numpy as np
from datetime import datetime
import re
import unicodedata
import os
pd.set_option('display.max_columns', None) #Para ver todas las columnas

In [2]:
dfh = pd.read_csv("Primary-data/Highest_victim_count.csv")
df5 = pd.read_csv("Primary-data/15_to_30_victim_count.csv")
df4 = pd.read_csv("Primary-data/5_to_14_victim_count.csv")
dfl = pd.read_csv("Primary-data/Lessthan_5_victim_count.csv")

# Paso 1:

## Comenzamos con un EDA inicial de los diferentes datasets

In [3]:
dfh.head()  #Mas primeras 5 filas del DataFrame (por defecto, pero puedes especificar un número).

,Name,Country,Years active,Proven victims,Possible victims,Notes
0,Luis Garavito,Colombia\r\nEcuador\r\nVenezuela,1992 to 1999,138,172–300+,"Child-murderer, torture-killer, and rapist kno..."
1,Pedro López,Colombia\r\nPeru\r\nEcuador,1969 to 1979,110,300+,"Child-murderer and rapist, known as ""The Monst..."
2,Javed Iqbal,Pakistan,1998 to 1999,100,100,"Child-murderer and rapist, known as ""Kukri"", I..."
3,Mikhail Popkov,Russia,1992 to 2010,83,83+,"Serial rapist-killer nicknamed ""The Werewolf"",..."
4,Daniel Camargo Barbosa,Colombia\r\nEcuador\r\nBrazil (alleged),1974 to 1986,72,180,"Child and woman murderer, believed to have pos..."


In [4]:
dfh.tail()  #Las últimas 5 filas del DataFrame (por defecto, pero puedes especificar un número).

,Name,Country,Years active,Proven victims,Possible victims,Notes
29,Ali Asghar Borujerdi,Ottoman Empire\r\n Iraq\r\n Iran,1907 to 1934,33,33,"Known as ""Asghar the Murderer"". Killed 33 youn..."
30,Vasili Komaroff,Soviet Union,1921 to 1923,33,33,"Known as ""The Wolf of Moscow""; horse trader wh..."
31,Fernando Hernández Leyva,Mexico,1982 to 1999,33,100+,Confessed to 100 murders and six kidnappings a...
32,Ramadan Abdel Rehim Mansour,Egypt,1999 to 2006,32,32+,"Gang leader known as al-Tourbini (""The Express..."
33,Radik Tagirov,Russia,2011 to 2012,32,32,Serial killer called Radik Tagirov who murdere...


In [5]:
dfh.sample()  #Una fila aleatoria del DataFrame (puedes especificar cuántas filas con `n`).

,Name,Country,Years active,Proven victims,Possible victims,Notes
22,Serhiy Tkach,Soviet Union\r\nUkraine,1984 to 2005,36,80–100,A former Ukrainian police criminal investigato...


In [6]:
dfh.columns  #Los nombres de las columnas.

Index(['Name', 'Country', 'Years active', 'Proven victims', 'Possible victims',
       'Notes'],
      dtype='object')

In [7]:
dfh.shape #Una tupla con el número de filas y columnas del DataFrame (filas, columnas).

(34, 6)

In [8]:
dfh.info()  #Info del DataFrame: número de filas, columnas y tipos de datos.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Name              34 non-null     object
 1   Country           34 non-null     object
 2   Years active      34 non-null     object
 3   Proven victims    34 non-null     int64 
 4   Possible victims  34 non-null     object
 5   Notes             34 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.7+ KB


In [9]:
dfh.dtypes #Tipo de datos de cada columna.

Name                object
Country             object
Years active        object
Proven victims       int64
Possible victims    object
Notes               object
dtype: object

In [10]:
dfh.describe().T #Suma, media, desviación estandar, mínimo/máximo y percentiles (valores numéricos).

,count,mean,std,min,25%,50%,75%,max
Proven victims,34.0,52.764706,24.601132,32.0,35.25,46.5,59.5,138.0


In [11]:
dfh.describe(include = 'object') #Suma, valores únicos... de tipo object.

,Name,Country,Years active,Possible victims,Notes
count,34,34,34,34,34
unique,34,20,34,32,34
top,Luis Garavito,United States,1992 to 1999,33,"Child-murderer, torture-killer, and rapist kno..."
freq,1,5,1,2,1


In [12]:
#Bucle con el que extraemos datos relevantes de cada columna:
columnas = dfh.columns

for col in columnas:
    print(f"\n🔹'{col}'\n")
    print(f"Tipo de dato: {dfh[col].dtypes}\n")
    print(f"Porcentaje de nulos: {dfh[col].isnull().sum()/dfh.shape[0]*100}\n")
    print(f"Suma de duplicados: {dfh[col].duplicated().sum()}\n")
    print(f"Valores únicos:")
    print(dfh[col].unique())
    print("-" * 100)


🔹'Name'

Tipo de dato: object

Porcentaje de nulos: 0.0

Suma de duplicados: 0

Valores únicos:
['Luis Garavito' 'Pedro López' 'Javed Iqbal' 'Mikhail Popkov'
 'Daniel Camargo Barbosa' 'Pedro Rodrigues Filho' 'Kampatimar Shankariya'
 'Yang Xinhai' 'Samuel Little' 'Giuseppe Greco' 'Andrei Chikatilo'
 'Anatoly Onoprienko' 'Ivan Maleshoff' 'Florisvaldo de Oliveira'
 'Willy Pickton' 'Gary Ridgway' 'Alexander Pichushkin' 'Wang Qiang'
 'Ahmad Suradji' 'Raman Raghav' 'Tiago Henrique Gomes da Rocha'
 'Moses Sithole' 'Serhiy Tkach' 'Gennady Mikhasevich'
 'Hadj Mohammed Mesfewi' 'Vera Renczi' 'Ted Bundy' 'Clementine Barnabet'
 'John Wayne Gacy' 'Ali Asghar Borujerdi' 'Vasili Komaroff'
 'Fernando Hernández Leyva' 'Ramadan Abdel Rehim Mansour' 'Radik Tagirov']
----------------------------------------------------------------------------------------------------

🔹'Country'

Tipo de dato: object

Porcentaje de nulos: 0.0

Suma de duplicados: 14

Valores únicos:
['Colombia\r\nEcuador\r\nVenezuela' 'Co

In [13]:
df5.head()  #Mas primeras 5 filas del DataFrame (por defecto, pero puedes especificar un número).

,Name,Country,Years active,Proven victims,Possible victims,Notes
0,Karl Denke,Germany,1900 to 1924,30,42+,Killed and cannibalized poor travelers and hom...
1,Francisco das Chagas Rodrigues de Brito,Brazil,1989 to 2003,30,42,"Pedophile who sexually abused, murdered and mu..."
2,Luis Gregorio Ramírez Maestre,Colombia,2010 to 2013,30,30,Killed motorists in various municipalities bef...
3,David Thabo Simelane,Swaziland,2000 to 2001,28,45,Sexually assaulted women he befriended in fore...
4,Zhang Jun,China,1993 to 2000,28,28,"Robbed 22 stores in several Chinese provinces,..."


In [14]:
df5.tail()  #Las últimas 5 filas del DataFrame (por defecto, pero puedes especificar un número).

,Name,Country,Years active,Proven victims,Possible victims,Notes
73,Elifasi Msomi,South Africa,1950s to 1956,15,15,Killed his victims with an axe or a knife in t...
74,Florencio Fernández,Argentina,1950s,15,15,"Stalked his victims, then would beat and bite ..."
75,Bai Baoshan,China,1983 to 1997,15,15,Robber who killed 15 people while robbing seve...
76,Maurizio Minghella,Italy,1978 to 2001,15,15,Killed five prostitutes; imprisoned and releas...
77,Alexander Labutkin,Soviet Union,1933 to 1935,15,15,"Known as the ""One-Armed Bandit""; killed people..."


In [15]:
df5.sample()  #Una fila aleatoria del DataFrame (puedes especificar cuántas filas con `n`).

,Name,Country,Years active,Proven victims,Possible victims,Notes
44,Asande Baninzi,South Africa,2001,18,18,"Killed 18 people in three months, including a ..."


In [16]:
df5.columns  #Los nombres de las columnas.

Index(['Name', 'Country', 'Years active', 'Proven victims', 'Possible victims',
       'Notes'],
      dtype='object')

In [17]:
df5.shape #Una tupla con el número de filas y columnas del DataFrame (filas, columnas).

(78, 6)

In [18]:
df5.info()  #Info del DataFrame: número de filas, columnas y tipos de datos.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Name              78 non-null     object
 1   Country           78 non-null     object
 2   Years active      78 non-null     object
 3   Proven victims    78 non-null     int64 
 4   Possible victims  76 non-null     object
 5   Notes             78 non-null     object
dtypes: int64(1), object(5)
memory usage: 3.8+ KB


In [19]:
df5.dtypes #Tipo de datos de cada columna.

Name                object
Country             object
Years active        object
Proven victims       int64
Possible victims    object
Notes               object
dtype: object

In [20]:
df5.describe().T #Suma, media, desviación estandar, mínimo/máximo y percentiles (valores numéricos).

,count,mean,std,min,25%,50%,75%,max
Proven victims,78.0,19.294872,4.099759,15.0,16.0,18.0,21.0,30.0


In [21]:
df5.describe(include = 'object') #Suma, valores únicos... de tipo object.

,Name,Country,Years active,Possible victims,Notes
count,78,78,78,76,78
unique,78,35,72,37,78
top,Karl Denke,United States,2005 to 2006,16,Killed and cannibalized poor travelers and hom...
freq,1,15,2,8,1


In [23]:
#Bucle con el que extraemos datos relevantes de cada columna:
columnas = df5.columns


for col in columnas:
    print(f"\n🔹'{col}'\n")
    print(f"Tipo de dato: {df5[col].dtypes}\n")
    print(f"Porcentaje de nulos: {df5[col].isnull().sum()/df5.shape[0]*100}\n")
    print(f"Suma de duplicados: {df5[col].duplicated().sum()}\n")
    print(f"Valores únicos:")
    print(df5[col].unique())
    print("-" * 100)


🔹'Name'

Tipo de dato: object

Porcentaje de nulos: 0.0

Suma de duplicados: 0

Valores únicos:
['Karl Denke' 'Francisco das Chagas Rodrigues de Brito'
 'Luis Gregorio Ramírez Maestre' 'David Thabo Simelane' 'Zhang Jun'
 'Cedric Maake' 'Mariam Soulakiotis' 'Bruce George Peter Lee'
 'The Stoneman' 'Juan Corona' 'Fritz Haarmann' 'Béla Kiss'
 'Majid Salek Mohammadi' 'Yvan Keller' 'Ronald Dominique'
 'Juan Fernando Hermosa' 'Earle Nelson' 'Mikhail Novosyolov'
 'Manuel Octavio Bermúdez' 'Patrick Kearney' 'William Bonin'
 'Vasile Tcaciuc' 'Yoo Young-chul' 'Francisco Guerrero Pérez'
 'Abdullah Shah' 'José Paz Bezerra' 'Bulelani Mabhayi' 'Mohan Kumar'
 'Alexander Spesivtsev' 'Larry Eyler' 'Surinder Koli' 'El Psicópata'
 'Sergei Ryakhovsky' 'Yevgeny Chuplinsky' 'M. Jaishankar' 'Ansis Kaupēns'
 'Vadim Ershov' 'Velaphi Ndlangamandla' 'Yavuz Yapıcıoğlu'
 'Paul John Knowles' 'Thierry Paulin' 'Randall Woodfield' 'Umesh Reddy'
 'Christopher Mhlengwa Zikode' 'Asande Baninzi' 'Huang Yong'
 'Pedro Pabl

In [24]:
df4.head()  #Mas primeras 5 filas del DataFrame (por defecto, pero puedes especificar un número).

,Name,Country,Years active,Proven victims,Possible victims,Notes
0,Carl Eugene Watts,United States,1974 to 1982,14,80+,Believed to have killed over 80 women in multi...
1,Philipp Tyurin,Soviet Union,1945 to 1946,14,29,"Known as the ""Leningrad Maniac"" and the ""Hellr..."
2,Zdzisław Marchwicki,Poland,1964 to 1970,14,21+,Also known as Vampire of Zagłębie. Killed 14 w...
3,Monster of Florence,Italy,1968 to 1985,14,16,Unidentified killer who shot couples in lovers...
4,Joachim Kroll,West Germany,1955 to 1976,14,NaN,"Known as the ""Ruhr Cannibal"" and ""The Duisburg..."


In [25]:
df4.tail()  #Las últimas 5 filas del DataFrame (por defecto, pero puedes especificar un número).

,Name,Country,Years active,Proven victims,Possible victims,Notes
159,Michel Peiry,Switzerland\r\nFrance\r\nUnited States\r\nYugo...,1981 to 1987,5,11,"Known as the ""Sadist of Romont""; Swiss soldier..."
160,Hubert Pilčík,Czechoslovakia,1948 to 1951,5,10+,Made money smuggling people across the Czechos...
161,Thomas Dillon,United States,1989 to 1992,5,NaN,Killed outdoorsmen in rural Ohio by sniping th...
162,Joe Metheny,United States,1976 to 1996,5,10,"Butchered his victims, then served them at BBQ..."
163,Sek Kim Wah,Singapore,30 June 1983 to 23 July 1983,5,-,"Sek Kim Wah, who was of Chinese Singaporean de..."


In [26]:
df4.sample()  #Una fila aleatoria del DataFrame (puedes especificar cuántas filas con `n`).

,Name,Country,Years active,Proven victims,Possible victims,Notes
8,Julio Pérez Silva,Chile,1998 to 2001,14,NaN,Known as The Psychopath from Alto Hospicio; ki...


In [27]:
df4.columns  #Los nombres de las columnas.

Index(['Name', 'Country', 'Years active', 'Proven victims', 'Possible victims',
       'Notes'],
      dtype='object')

In [28]:
df4.shape #Una tupla con el número de filas y columnas del DataFrame (filas, columnas).

(164, 6)

In [29]:
df4.info()  #Info del DataFrame: número de filas, columnas y tipos de datos.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Name              164 non-null    object
 1   Country           164 non-null    object
 2   Years active      164 non-null    object
 3   Proven victims    164 non-null    object
 4   Possible victims  98 non-null     object
 5   Notes             164 non-null    object
dtypes: object(6)
memory usage: 7.8+ KB


In [30]:
df4.dtypes #Tipo de datos de cada columna.

Name                object
Country             object
Years active        object
Proven victims      object
Possible victims    object
Notes               object
dtype: object

In [31]:
df4.describe() #Suma, media, desviación estandar, mínimo/máximo y percentiles (valores numéricos).

,Name,Country,Years active,Proven victims,Possible victims,Notes
count,164,164,164,164,98,164
unique,164,48,150,14,51,164
top,Carl Eugene Watts,United States,1984,11,11,Believed to have killed over 80 women in multi...
freq,1,61,2,24,9,1


In [32]:
df4.describe(include = 'object') #Suma, valores únicos... de tipo object.

,Name,Country,Years active,Proven victims,Possible victims,Notes
count,164,164,164,164,98,164
unique,164,48,150,14,51,164
top,Carl Eugene Watts,United States,1984,11,11,Believed to have killed over 80 women in multi...
freq,1,61,2,24,9,1


In [33]:
#Bucle con el que extraemos datos relevantes de cada columna:
columnas = df4.columns

for col in columnas:
    print(f"\n🔹'{col}'\n")
    print(f"Tipo de dato: {df4[col].dtypes}\n")
    print(f"Porcentaje de nulos: {df4[col].isnull().sum()/df4.shape[0]*100}\n")
    print(f"Suma de duplicados: {df4[col].duplicated().sum()}\n")
    print(f"Valores únicos:")
    print(df4[col].unique())
    print("-" * 100)


🔹'Name'

Tipo de dato: object

Porcentaje de nulos: 0.0

Suma de duplicados: 0

Valores únicos:
['Carl Eugene Watts' 'Philipp Tyurin' 'Zdzisław Marchwicki'
 'Monster of Florence' 'Joachim Kroll' 'Arthur Shawcross' 'The Doodler'
 'Marcelo Costa de Andrade' 'Julio Pérez Silva' 'Sergey Shipilov'
 'Abdufatto Zamanov' 'Jeong Nam-gyu' 'Amir Qayyum' 'Jake Bird'
 'Belle Gunness' 'Cleveland Torso Murderer' 'Kaspars Petrovs'
 'Richard Ramirez' 'Sleepy Hollow Killer' 'Peter Sutcliffe'
 'Francisco Antonio Laureana' 'Tamara Samsonova' 'Herbert Mullin'
 'Boston Strangler' 'Lorenzo Gilyard' 'Joseph James DeAngelo'
 'Vladimir Storozhenko' 'Vasiliy Kulik' 'Nikolai Dudin' 'Denis Pischikov'
 'Nikolay Shubin' 'Johannes Mashiane' 'Mukosi Freddy Mulaudzi'
 'Thozamile Taki' 'Rainbow Maniac' 'Naceur Damergi' 'Adolf Seefeldt'
 'Donald Henry Gaskins' 'William Suff' 'Vladimir Romanov' 'Maury Travis'
 'Dennis Nilsen' 'Kenneth Bianchi' 'Charles Sobhraj' 'Abdallah al-Hubal'
 'Nikolai Shestakov' 'Anatoly Sedykh' 'J

In [34]:
dfl.head()  #Mas primeras 5 filas del DataFrame (por defecto, pero puedes especificar un número).

,Name,Country,Years active,Proven victims,Possible victims,Notes
0,Charlie Brandt,United States,1971 to 2004,4,29,Committed suicide by hanging after murdering h...
1,Robert Black,United Kingdom\r\nIreland (suspected)\r\nWest ...,1981 to 1986,4,18+,"Convicted of kidnapping, raping and murdering ..."
2,Max Gufler,Austria,1946 to 1958,4,18,"Poisoned and drowned four women, but suspected..."
3,Ernesto Picchioni,Italy,1949 and earlier,4,16,Murdered people who approached his home; died ...
4,Baekuni,Indonesia,1993 to 2010,4,14,Pedophile who raped and killed young boys; ini...


In [35]:
dfl.tail()  #Las últimas 5 filas del DataFrame (por defecto, pero puedes especificar un número).

,Name,Country,Years active,Proven victims,Possible victims,Notes
24,Leonarda Cianciulli,Italy,1939 to 1940,3,-,"Murdered three women in the town of Correggio,..."
25,Özkan Zengin,Turkey,2008,3,4,Slit the throats of three confirmed victims be...
26,Albert Fish,United States,1924 to 1932,3,9-100+,American serial killer and child rapist who co...
27,Heriberto Seda,United States,1990 to 1996,3,8+,Considered to be a copycat of the Zodiac Kille...
28,John Martin,United Kingdom,1995,3,3+,"(Birth name John Martin Scripps, alias Simon J..."


In [36]:
dfl.sample()  #Una fila aleatoria del DataFrame (puedes especificar cuántas filas con `n`).

,Name,Country,Years active,Proven victims,Possible victims,Notes
19,Stephen Griffiths,United Kingdom,2009 to 2010,3,14,"Known to have killed three prostitutes, but cl..."


In [37]:
dfl.columns  #Los nombres de las columnas.

Index(['Name', 'Country', 'Years active', 'Proven victims', 'Possible victims',
       'Notes'],
      dtype='object')

In [38]:
dfl.shape #Una tupla con el número de filas y columnas del DataFrame (filas, columnas).

(29, 6)

In [39]:
dfl.info()  #Info del DataFrame: número de filas, columnas y tipos de datos.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Name              29 non-null     object
 1   Country           29 non-null     object
 2   Years active      29 non-null     object
 3   Proven victims    29 non-null     int64 
 4   Possible victims  29 non-null     object
 5   Notes             29 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.5+ KB


In [40]:
dfl.dtypes #Tipo de datos de cada columna.

Name                object
Country             object
Years active        object
Proven victims       int64
Possible victims    object
Notes               object
dtype: object

In [41]:
dfl.describe() #Suma, media, desviación estandar, mínimo/máximo y percentiles (valores numéricos).

,Proven victims
count,29.000000
mean,3.379310
std,0.493804
min,3.000000
25%,3.000000
50%,3.000000
75%,4.000000
max,4.000000


In [42]:
dfl.describe(include = 'object') #Suma, valores únicos... de tipo object.

,Name,Country,Years active,Possible victims,Notes
count,29,29,29,29,29
unique,29,12,28,22,29
top,Charlie Brandt,United States,1985 to 1986,12,Committed suicide by hanging after murdering h...
freq,1,11,2,3,1


In [43]:
#Bucle con el que extraemos datos relevantes de cada columna:
columnas = dfl.columns

for col in columnas:
    print(f"\n🔹'{col}'\n")
    print(f"Tipo de dato: {dfl[col].dtypes}\n")
    print(f"Porcentaje de nulos: {dfl[col].isnull().sum()/dfl.shape[0]*100}\n")
    print(f"Suma de duplicados: {dfl[col].duplicated().sum()}\n")
    print(f"Valores únicos:")
    print(dfl[col].unique())
    print("-" * 100)


🔹'Name'

Tipo de dato: object

Porcentaje de nulos: 0.0

Suma de duplicados: 0

Valores únicos:
['Charlie Brandt' 'Robert Black' 'Max Gufler' 'Ernesto Picchioni'
 'Baekuni' 'Igor Chernat' 'Ricky Lee Green' 'Robert Hicks Murray'
 'Dallen Bounds' 'Stephen Port' 'Robert Maudsley' 'Sharon Kinne'
 'Ted Kaczynski' 'Robert Ben Rhoades' 'Peter Tobin' 'Pedro Padilla Flores'
 'Billy Glaze' 'Bertha Gifford' 'Bernhard Prigan' 'Stephen Griffiths'
 'Koos Hertogs' 'Chisako Kakehi' 'Dorothea Puente' 'Patrick Mackay'
 'Leonarda Cianciulli' 'Özkan Zengin' 'Albert Fish' 'Heriberto Seda'
 'John Martin']
----------------------------------------------------------------------------------------------------

🔹'Country'

Tipo de dato: object

Porcentaje de nulos: 0.0

Suma de duplicados: 17

Valores únicos:
['United States'
 'United Kingdom\r\nIreland (suspected)\r\nWest Germany (suspected)\r\nNetherlands (suspected)\r\nFrance (suspected)'
 'Austria' 'Italy' 'Indonesia' 'Soviet Union' 'United Kingdom' 'Mexico'

## A partir de aqui hacemos varias comprobaciones para asegurarnos que los datasets son iguales y se pueden concatenar

In [44]:
print(dfh.shape)
dfh['Name'].nunique()

(34, 6)


34

In [45]:
print(df5.shape)
df5['Name'].nunique()

(78, 6)


78

In [46]:
print(df4.shape)
df4['Name'].nunique()

(164, 6)


164

In [47]:
print(dfl.shape)
dfl['Name'].nunique()

(29, 6)


29

# Paso 2:

## Pasamos a la limpieza y normalizacion de los datos

In [48]:
# Concatenar los df:
dfc = pd.concat([dfh, df5, df4, dfl], ignore_index=True)

In [49]:
# Revisar duplicados
dfc.duplicated().sum()

0

In [50]:
# Cambiar los espacios por "_":
dfc.columns #Me da todas las columnas

# Diccionario vacío:
col_f = {} 

# Itera dfc.columns y crea un diccionario clave : valor con el cambio.
for columna in dfc.columns:
    col_f[columna] = columna.replace(" ", "_")

# Rename:
dfc.rename(columns = col_f, inplace = True)

In [51]:
dfc.columns

Index(['Name', 'Country', 'Years_active', 'Proven_victims', 'Possible_victims',
       'Notes'],
      dtype='object')

In [52]:
dfc['Years_active'].to_frame().T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304
Years_active,1992 to 1999,1969 to 1979,1998 to 1999,1992 to 2010,1974 to 1986,1967 to 2003,1977 to 1978,2000 to 2003,1970 to 2005,1981 to 1983,1978 to 1990,1989 to 1996,1934 to 1935,1982,1983 to 2002,1982 to 2000,1992 to 2006,1995 to 2003,1986 to 1997,1965 to 1968,2011 to 2014,1994 to 1995,1984 to 2005,1971 to 1985,1906 and earlier,1920 to 1930,1974 to 1978,1911,1972 to 1978,1907 to 1934,1921 to 1923,1982 to 1999,1999 to 2006,2011 to 2012,1900 to 1924,1989 to 2003,2010 to 2013,2000 to 2001,1993 to 2000,1996 to 1997,1939 to 1951,1973 to 1979,1985 to 1989,1971,1918 to 1924,1912 to 1916,1981 to 1985,1989 to 2006,1997 to 2006,1991 to 1992,1926 to 1927,1977 to 1995,1999 to 2003,1965 to 1977,1979 to 1980,1917 to 1935,2003 to 2004,1880 to 1908,1990s,1960s to 1970s,2007 to 2012,2005 to 2009,1991 to 1996,1982 to 1984,2005 to 2006,1986 to 1996,1988 to 1993,1998 to 2006,2008 to 2011,1920 to 1926,1992 to 1995,1998,1994 to 2002,1974,1984 to 1987,1979 to 1981,1996 to 2002,1994 to 1995,2001,2001 to 2003,2005 to 2006,2004,1992 to 2002,1997 to 1998,2002 to 2010,1971 to 1983,2004,1987 to 2001,1996 to 1997,2000 to 2001,1978 to 1991,1987 to 1988,1950s,2005,2008 to 2009,1975 to 1998,1948 to 1980,1969 to 1982,1974 to 1987,2003 to 2004,1986 to 1994,1997 to 1999,2008 to 2015,1911,1980 to 1983,1986 to 1999,1987 to 1998,1950s to 1956,1950s,1983 to 1997,1978 to 2001,1933 to 1935,1974 to 1982,1945 to 1946,1964 to 1970,1968 to 1985,1955 to 1976,1972 to 1989,1974 to 1975,1991,1998 to 2001,1995 to 1999,2002 to 2004,2004 to 2006,2005,1930s to 1947,1900? to 1908?,1934 to 1938,2003,1984 to 1985,1990s to 2007,1975 to 1980,1974 to 1975,1995 to 2015,1972 to 1973,1962 to 1964,1977 to 1993,1979 to 1986,1978 to 1981,1984 to 1986,1987 to 2002,2002 to 2003,2004 to 2006,1982 to 1989,1990 to 2006,2007,2007 to 2008,1980s to 1988,1908 to 1935,1953 to 1982,1986 to 1992,1991 to 2005,2000 to 2002,1978 to 1983,1977 to 1978,1975 to 1976,1990 to 1998,1975,1998 to 2003,1980 to 1981,1985,c.1900 to 1912,1990 to 1996,1992 to 1999,1986 to 1992,late 1990s to 2006,2008 to 2012,1974 to 1992,1964 to 1975,1991 to 1992,1927 to 1954,1980 to 1981,1915 to 1919,2003 to 2005,2007 to 2009,1993 to 1999,1987 to 1994,1971,1992 to 1995,1938 to 1942,2008 and earlier,1997 to 1998,2008,1998 to 2002,1933 to 1936,2000 to 2003,1998 to 2003,1946 to 1947,1983,1996 to 2011,1979 to 1986,1985 to 2007,1999,1984,1990 to 1997,2004 to 2008,1964 to 1973,1974 to 2004,1997 to present,1992 to 1999,1905 to 1908,1990 to 1994,1977 to 1979,1995 to 1998,2004 to 2012,2006 to 2008,1996 to 1998,1991 to 1992,1980 to 1982,1974 to 2006,1986 to 1989,1913 to 1930,1974 to 1978,1986 to 1994,1984 to 1992,1989 to 1993,1913 to 1920,1966 to 1992,1981 to 1996,1958 to 1983,2009 to 2012,1988 to 1989,2015 to 2016,1996 to 2000,2000s,2000s,1958 to 1963,1977 to 1980,1928? to 1946,1990 to 1995,1971 to 1979,1982 to 1983,1980 to 1988,1984,1918 to 1919,1990

In [53]:
# Dividir 'Years_active' en 'Start Stop' y modificar la columna 'Years_active' (total años en activo)
# Crear la columna 'Start_year' a partir de 'Years_active': Eliminando el 'to' y lo que va después.
dfc['Start_year'] = dfc['Years_active'].str.split(" to ").str[0]

In [54]:
dfc['Start_year'].unique()

array(['1992', '1969', '1998', '1974', '1967', '1977', '2000', '1970',
       '1981', '1978', '1989', '1934', '1982', '1983', '1995', '1986',
       '1965', '2011', '1994', '1984', '1971', '1906 and earlier', '1920',
       '1911', '1972', '1907', '1921', '1999', '1900', '2010', '1993',
       '1996', '1939', '1973', '1985', '1918', '1912', '1997', '1991',
       '1926', '1979', '1917', '2003', '1880', '1990s', '1960s', '2007',
       '2005', '1988', '2008', '2001', '2004', '2002', '1987', '1950s',
       '1975', '1948', '1980', '1933', '1945', '1964', '1968', '1955',
       '1930s', '1900?', '1962', '1990', '1980s', '1908', '1953',
       'c.1900', 'late 1990s', '1927', '1915', '1938', '2008 and earlier',
       '1946', '1905', '2006', '1913', '1966', '1958', '2009', '2015',
       '2000s', '1928?', '1980 and earlier', '1960', '2016', '1976',
       '2014', '1957', '30 June 1983', '1949 and earlier',
       '1912 and earlier', '1909', '1947', '1924'], dtype=object)

In [55]:
# Las columnas son una caca y tuve que preguntarle a chatgpt qué hacer, después de provar mil cosas me dió este código:
# Extraer los cuatro primero dígitos de la cadena:
dfc['Start_year'] = dfc['Start_year'].str.extract(r'(\d{4})')

# Convertir la columna a numérico:
dfc['Start_year'] = pd.to_numeric(dfc['Start_year'], errors='coerce')

In [56]:
# Revisar que todo ok:
dfc['Start_year'].to_frame().T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304
Start_year,1992,1969,1998,1992,1974,1967,1977,2000,1970,1981,1978,1989,1934,1982,1983,1982,1992,1995,1986,1965,2011,1994,1984,1971,1906,1920,1974,1911,1972,1907,1921,1982,1999,2011,1900,1989,2010,2000,1993,1996,1939,1973,1985,1971,1918,1912,1981,1989,1997,1991,1926,1977,1999,1965,1979,1917,2003,1880,1990,1960,2007,2005,1991,1982,2005,1986,1988,1998,2008,1920,1992,1998,1994,1974,1984,1979,1996,1994,2001,2001,2005,2004,1992,1997,2002,1971,2004,1987,1996,2000,1978,1987,1950,2005,2008,1975,1948,1969,1974,2003,1986,1997,2008,1911,1980,1986,1987,1950,1950,1983,1978,1933,1974,1945,1964,1968,1955,1972,1974,1991,1998,1995,2002,2004,2005,1930,1900,1934,2003,1984,1990,1975,1974,1995,1972,1962,1977,1979,1978,1984,1987,2002,2004,1982,1990,2007,2007,1980,1908,1953,1986,1991,2000,1978,1977,1975,1990,1975,1998,1980,1985,1900,1990,1992,1986,1990,2008,1974,1964,1991,1927,1980,1915,2003,2007,1993,1987,1971,1992,1938,2008,1997,2008,1998,1933,2000,1998,1946,1983,1996,1979,1985,1999,1984,1990,2004,1964,1974,1997,1992,1905,1990,1977,1995,2004,2006,1996,1991,1980,1974,1986,1913,1974,1986,1984,1989,1913,1966,1981,1958,2009,1988,2015,1996,2000,2000,1958,1977,1928,1990,1971,1982,1980,1984,1918,1990,1965,2010,2007,1980,1964,1991,1990,1980,1986,1979,1997,1960,2003,1981,2002,2002,1989,2016,1976,1977,1980,1950,2005,1990,1986,1989,2000,2014,1977,1978,1977,1962,1957,1915,2006,1981,1948,1989,1976,1983,1971,1981,1946,1949,1993,1985,1985,1912,1999,2014,1974,1960,1978,1989,1991,1986,1986,1909,1947,2009,1979,2007,1982,1973,1939,2008,1924,1990,1995


In [57]:
# Para 'End_year' también era un lío así que me ayudó de nuevo chatgpt.
# Extraer todos los números de 4 dígitos en una lista para cada registro y tomar solamente el último, si no hay devuelve nulo.
dfc['End_year'] = dfc['Years_active'].str.findall(r'(\d{4})').apply(lambda x: x[-1] if x else np.nan)

# Convertir la columna a numérico
dfc['End_year'] = pd.to_numeric(dfc['End_year'], errors='coerce')

In [58]:
# Revisar again que todo ok:
dfc['End_year'].to_frame().T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304
End_year,1999,1979,1999,2010,1986,2003,1978,2003,2005,1983,1990,1996,1935,1982,2002,2000,2006,2003,1997,1968,2014,1995,2005,1985,1906,1930,1978,1911,1978,1934,1923,1999,2006,2012,1924,2003,2013,2001,2000,1997,1951,1979,1989,1971,1924,1916,1985,2006,2006,1992,1927,1995,2003,1977,1980,1935,2004,1908,1990,1970,2012,2009,1996,1984,2006,1996,1993,2006,2011,1926,1995,1998,2002,1974,1987,1981,2002,1995,2001,2003,2006,2004,2002,1998,2010,1983,2004,2001,1997,2001,1991,1988,1950,2005,2009,1998,1980,1982,1987,2004,1994,1999,2015,1911,1983,1999,1998,1956,1950,1997,2001,1935,1982,1946,1970,1985,1976,1989,1975,1991,2001,1999,2004,2006,2005,1947,1908,1938,2003,1985,2007,1980,1975,2015,1973,1964,1993,1986,1981,1986,2002,2003,2006,1989,2006,2007,2008,1988,1935,1982,1992,2005,2002,1983,1978,1976,1998,1975,2003,1981,1985,1912,1996,1999,1992,2006,2012,1992,1975,1992,1954,1981,1919,2005,2009,1999,1994,1971,1995,1942,2008,1998,2008,2002,1936,2003,2003,1947,1983,2011,1986,2007,1999,1984,1997,2008,1973,2004,1997,1999,1908,1994,1979,1998,2012,2008,1998,1992,1982,2006,1989,1930,1978,1994,1992,1993,1920,1992,1996,1983,2012,1989,2016,2000,2000,2000,1963,1980,1946,1995,1979,1983,1988,1984,1919,1996,1987,2017,2018,1980,1971,1999,1990,2010,1986,1981,1997,1980,2016,2004,2002,2018,1990,2018,1977,1978,1999,1950,2006,2015,1990,1990,2006,2015,1994,2002,1980,1977,2009,1929,2006,1987,1951,1992,1996,1983,2004,1986,1958,1949,2010,1986,1986,1912,1999,2015,1978,1964,1996,1990,2006,1986,1987,1928,1952,2010,1980,2013,1988,1975,1940,2008,1932,1996,1995


In [59]:
# Comparar las dos columnas:
dfc['Start_year'].to_frame().T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304
Start_year,1992,1969,1998,1992,1974,1967,1977,2000,1970,1981,1978,1989,1934,1982,1983,1982,1992,1995,1986,1965,2011,1994,1984,1971,1906,1920,1974,1911,1972,1907,1921,1982,1999,2011,1900,1989,2010,2000,1993,1996,1939,1973,1985,1971,1918,1912,1981,1989,1997,1991,1926,1977,1999,1965,1979,1917,2003,1880,1990,1960,2007,2005,1991,1982,2005,1986,1988,1998,2008,1920,1992,1998,1994,1974,1984,1979,1996,1994,2001,2001,2005,2004,1992,1997,2002,1971,2004,1987,1996,2000,1978,1987,1950,2005,2008,1975,1948,1969,1974,2003,1986,1997,2008,1911,1980,1986,1987,1950,1950,1983,1978,1933,1974,1945,1964,1968,1955,1972,1974,1991,1998,1995,2002,2004,2005,1930,1900,1934,2003,1984,1990,1975,1974,1995,1972,1962,1977,1979,1978,1984,1987,2002,2004,1982,1990,2007,2007,1980,1908,1953,1986,1991,2000,1978,1977,1975,1990,1975,1998,1980,1985,1900,1990,1992,1986,1990,2008,1974,1964,1991,1927,1980,1915,2003,2007,1993,1987,1971,1992,1938,2008,1997,2008,1998,1933,2000,1998,1946,1983,1996,1979,1985,1999,1984,1990,2004,1964,1974,1997,1992,1905,1990,1977,1995,2004,2006,1996,1991,1980,1974,1986,1913,1974,1986,1984,1989,1913,1966,1981,1958,2009,1988,2015,1996,2000,2000,1958,1977,1928,1990,1971,1982,1980,1984,1918,1990,1965,2010,2007,1980,1964,1991,1990,1980,1986,1979,1997,1960,2003,1981,2002,2002,1989,2016,1976,1977,1980,1950,2005,1990,1986,1989,2000,2014,1977,1978,1977,1962,1957,1915,2006,1981,1948,1989,1976,1983,1971,1981,1946,1949,1993,1985,1985,1912,1999,2014,1974,1960,1978,1989,1991,1986,1986,1909,1947,2009,1979,2007,1982,1973,1939,2008,1924,1990,1995


Al comparar los datos de 'Years_active', 'Start_year' y 'End_year' hay que cambiar ciertas entradas en 'End_year':
- A 1959 en_ 92, 108 y 27. Ya que estuvieron activos en 1950s.
- A 1999 en: 58 y 242. Ya que estuvieron activos en 1990s.
- A 2009 en: 224 y 225. Ya que estuvieron activos en 2000s.
- En el id 198 indica "to present", ¿asumimos que la fecha es 2018 ya que es el último año de actividad?

In [60]:
# Ver cuál es el último año:
unique_values = np.sort(dfc['End_year'].unique())[::-1]
print(unique_values)

[2018 2017 2016 2015 2014 2013 2012 2011 2010 2009 2008 2007 2006 2005
 2004 2003 2002 2001 2000 1999 1998 1997 1996 1995 1994 1993 1992 1991
 1990 1989 1988 1987 1986 1985 1984 1983 1982 1981 1980 1979 1978 1977
 1976 1975 1974 1973 1971 1970 1968 1964 1963 1958 1956 1954 1952 1951
 1950 1949 1947 1946 1942 1940 1938 1936 1935 1934 1932 1930 1929 1928
 1927 1926 1924 1923 1920 1919 1916 1912 1911 1908 1906]


In [61]:
#Cambios en 'End_year'.
# A 1959 en_ 92, 108 y 27. Ya que estuvieron activos en 1950s.
dfc.loc[[92, 108, 27], 'End_year'] = 1959

# A 1999 en: 58 y 242. Ya que estuvieron activos en 1990s.
dfc.loc[[58, 242], 'End_year'] = 1999

# A 2009 en: 24 y 25. Ya que estuvieron activos en 2000s.
dfc.loc[[224, 225], 'End_year'] = 2009

# En el id 198 indica "to present", ¿asumimos que la fecha es 2018 ya que es el último año de actividad?
dfc.loc[[198], 'End_year'] = 2018

In [62]:
# Columna nueva con años en activo, nuevo nombre para que no se solape:
# Añadimos el +1 para que los que llevan solo un año en activo no se quede 0 y para que el primer año también cuente en el resto.
dfc['Años_activo'] = dfc['End_year'] - dfc['Start_year'] + 1

In [63]:
dfc['Años_activo'].unique()

array([ 8, 11,  2, 19, 13, 37,  4, 36,  3,  1, 20, 15,  9, 12, 22,  5, 49,
        7, 28, 18, 25, 10, 29,  6, 14, 24, 33, 21, 17, 16, 30, 23, 31, 27,
       26, 53, 34])

In [64]:
# Eliminar 'Years_active' para después cambiar 'Años_activo':
dfc.drop('Years_active', axis=1, inplace=True)

In [65]:
# Cambio de nombre:
dfc.rename(columns = {'Años_activo': 'Years_active'}, inplace = True)

Vamos ahora con 'Proven_victims'

In [66]:
dfc['Proven_victims'].unique()

array([138, 110, 100, 83, 72, 71, 70, 67, 60, 58, 53, 52, 50, 49, 48, 45,
       42, 41, 39, 38, 36, 35, 33, 32, 30, 28, 27, 26, 25, 24, 23, 22, 21,
       20, 19, 18, 17, 16, 15, '14', '13', '12', '11–16', '11', '10',
       '9–13', '9', '8', '8–15', '8–11+', '7', '6', '5', 4, 3],
      dtype=object)

In [67]:
# Hago ua función para cambiar los valores que "no sirven" porque intentando hacerlo con código la liaba siempre.
def intervalos(Proven_victims):
    if Proven_victims == '11–16':
        return "11"
    elif Proven_victims == "9–13":
        return "9"
    elif Proven_victims == "8–15":
        return "8"
    elif Proven_victims == "8–11+":
        return "8"
    else:
        return Proven_victims

In [68]:
# Aplica la función:
dfc['Proven_victims'] = dfc['Proven_victims'].apply(intervalos)

In [69]:
# Reviso:
dfc['Proven_victims'].unique()

array([138, 110, 100, 83, 72, 71, 70, 67, 60, 58, 53, 52, 50, 49, 48, 45,
       42, 41, 39, 38, 36, 35, 33, 32, 30, 28, 27, 26, 25, 24, 23, 22, 21,
       20, 19, 18, 17, 16, 15, '14', '13', '12', '11', '10', '9', '8',
       '7', '6', '5', 4, 3], dtype=object)

In [70]:
# Cambio a int:
dfc['Proven_victims'] = dfc['Proven_victims'].astype(int)

In [71]:
# Reviso again:
dfc['Proven_victims'].unique()

array([138, 110, 100,  83,  72,  71,  70,  67,  60,  58,  53,  52,  50,
        49,  48,  45,  42,  41,  39,  38,  36,  35,  33,  32,  30,  28,
        27,  26,  25,  24,  23,  22,  21,  20,  19,  18,  17,  16,  15,
        14,  13,  12,  11,  10,   9,   8,   7,   6,   5,   4,   3])

Nos encontramos que en los registros de la columna country aparecen listas de paises separados por saltos de fila. Tableau no puede interpretar estos datos correctamente 
por lo que trás hablar con la PO tomamos la decision de hacer un .explode por filas y quedarnos con la primera.

In [72]:
print(dfc['Country'].unique())

['Colombia\r\nEcuador\r\nVenezuela' 'Colombia\r\nPeru\r\nEcuador'
 'Pakistan' 'Russia' 'Colombia\r\nEcuador\r\nBrazil (alleged)' 'Brazil'
 'India' 'China' 'United States' 'Italy' 'Soviet Union'
 'Soviet Union\r\nUkraine' 'Canada' 'Indonesia' 'South Africa' 'Morocco'
 'Kingdom of Romania\r\nYugoslavia\r\nHungary (alleged)'
 'Ottoman Empire\r\n Iraq\r\n Iran' 'Mexico' 'Egypt' 'Germany' 'Colombia'
 'Swaziland' 'Greece' 'United Kingdom' 'Austria-Hungary' 'Iran'
 'France\r\nGermany (suspected)\r\nSwitzerland (suspected)' 'Ecuador'
 'United States\r\nCanada' 'Soviet Union\r\nRussia\r\nTajikistan'
 'Romania' 'South Korea' 'Afghanistan' 'Costa Rica'
 'Soviet Union\r\nRussia' 'Latvia' 'Turkey' 'France' 'Peru' 'Ukraine'
 'France\r\nBelgium' 'Spain' 'Venezuela' 'United States\r\nMexico'
 'Argentina' 'Poland' 'West Germany' 'Chile' 'Tunisia'
 'Thailand\r\nNepal\r\nIndia\r\nMalaysia' 'Yemen' 'Japan' 'Australia'
 'Austria\r\nUnited States\r\nCzechoslovakia' 'Bangladesh' 'Belgium'
 'Soviet Union\r\nR

In [ ]:
#Separamos los países por salto de línea, explotamos filas y limpiamos espacios delante y detrás
dfc['Country'] = dfc['Country'].str.split(r'\r\n')
dfc = dfc.explode('Country')
dfc['Country'] = dfc['Country'].str.strip()
#eliminamos duplicados
dfc = dfc.drop_duplicates(subset='Name', keep='first').reset_index(drop=True)


In [ ]:
#comprobamos que los duplicados se han eliminado correctamente.
duplicates = dfc.duplicated(subset='Name').sum()
print(f'Duplicados encontrados: {duplicates}')


Duplicados encontrados: 0


In [75]:
# comprobamos que se ha ejecutado correctamente
print(sorted(dfc['Country'].unique()))
print(dfc['Country'].isnull().sum())
dfc['Country'].value_counts()



['Afghanistan', 'Allied-occupied Germany', 'Argentina', 'Australia', 'Austria', 'Austria-Hungary', 'Bangladesh', 'Belarus', 'Belgium', 'Brazil', 'Canada', 'Chile', 'China', 'Colombia', 'Costa Rica', 'Cyprus', 'Czechoslovakia', 'Denmark', 'East Germany', 'Ecuador', 'Egypt', 'France', 'German Empire', 'Germany', 'Greece', 'India', 'Indonesia', 'Iran', 'Italy', 'Japan', 'Kingdom of Romania', 'Latvia', 'Mexico', 'Morocco', 'Netherlands', 'Ottoman Empire', 'Pakistan', 'Peru', 'Poland', 'Romania', 'Russia', 'Singapore', 'South Africa', 'South Korea', 'Soviet Union', 'Spain', 'Swaziland', 'Switzerland', 'Thailand', 'Tunisia', 'Turkey', 'Ukraine', 'United Kingdom', 'United States', 'Venezuela', 'West Germany', 'Yemen']
0


Country
United States              95
Russia                     21
Soviet Union               21
South Africa               19
United Kingdom             13
India                      10
China                      10
Brazil                      8
France                      8
Colombia                    7
Italy                       6
Canada                      6
Australia                   5
South Korea                 4
Spain                       4
Japan                       4
Turkey                      4
Mexico                      4
Germany                     3
Iran                        3
Argentina                   3
Indonesia                   3
West Germany                2
Latvia                      2
Poland                      2
Bangladesh                  2
Belgium                     2
Ukraine                     2
Austria                     2
Costa Rica                  2
Pakistan                    2
Ottoman Empire              1
Allied-occupied Germany     1
Si

In [76]:
dfc.head(5)

,Name,Country,Proven_victims,Possible_victims,Notes,Start_year,End_year,Years_active
0,Luis Garavito,Colombia,138,172–300+,"Child-murderer, torture-killer, and rapist kno...",1992,1999,8
1,Pedro López,Colombia,110,300+,"Child-murderer and rapist, known as ""The Monst...",1969,1979,11
2,Javed Iqbal,Pakistan,100,100,"Child-murderer and rapist, known as ""Kukri"", I...",1998,1999,2
3,Mikhail Popkov,Russia,83,83+,"Serial rapist-killer nicknamed ""The Werewolf"",...",1992,2010,19
4,Daniel Camargo Barbosa,Colombia,72,180,"Child and woman murderer, believed to have pos...",1974,1986,13


Vemos que tras la limpieza nuestro dataset son pocos registros pasa analizar y añadimos dos csv más para completar

In [77]:
df_victims = pd.read_csv("Primary-data/serial_killers_by_victims.csv", sep=";")
df_profile = pd.read_csv("Primary-data/profile_killers.csv", sep=";")

In [78]:
df_profile.info()
df_victims.info()
dfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711 entries, 0 to 710
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              711 non-null    object 
 1   Nicknames         469 non-null    object 
 2   Age               243 non-null    float64
 3   Born Date         444 non-null    object 
 4   Born Location     420 non-null    object 
 5   Died Date         308 non-null    object 
 6   Died Location     229 non-null    object 
 7   Conviction        500 non-null    object 
 8   Criminal Penalty  580 non-null    object 
 9   Date Apprehended  543 non-null    object 
dtypes: float64(1), object(9)
memory usage: 55.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 757 entries, 0 to 756
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              757 non-null    object 
 1   Country           757 non-nu

In [79]:
print("Columnas en df_victims:")
print(df_victims.columns)

print("Columnas en df_profile:")
print(df_profile.columns)


Columnas en df_victims:
Index(['Name', 'Country', 'Start year', 'End year', 'Proven victims',
       'Possible victims', 'Notes'],
      dtype='object')
Columnas en df_profile:
Index(['Name', 'Nicknames', 'Age', 'Born Date', 'Born Location', 'Died Date',
       'Died Location', 'Conviction', 'Criminal Penalty', 'Date Apprehended'],
      dtype='object')


In [ ]:
#Unimos los dos csv nuevos con un outer para mantener todas las columnas.
df_new_killers = df_victims.merge(df_profile, on='Name', how='outer')

In [81]:
df_new_killers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852 entries, 0 to 851
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              852 non-null    object 
 1   Country           757 non-null    object 
 2   Start year        756 non-null    float64
 3   End year          751 non-null    float64
 4   Proven victims    757 non-null    float64
 5   Possible victims  757 non-null    float64
 6   Notes             754 non-null    object 
 7   Nicknames         469 non-null    object 
 8   Age               243 non-null    float64
 9   Born Date         444 non-null    object 
 10  Born Location     420 non-null    object 
 11  Died Date         308 non-null    object 
 12  Died Location     229 non-null    object 
 13  Conviction        500 non-null    object 
 14  Criminal Penalty  580 non-null    object 
 15  Date Apprehended  543 non-null    object 
dtypes: float64(5), object(11)
memory usage: 106.

In [83]:
#Normalizamos los nombres de las columnas con el mismo codigo que hicimos anteriormente con el dataframe DFC
df_new_killers.columns
col_f = {} 
for columna in df_new_killers.columns:
    col_f[columna] = columna.replace(" ", "_")

df_new_killers.rename(columns = col_f, inplace = True)

print(df_new_killers.columns)

Index(['Name', 'Country', 'Start_year', 'End_year', 'Proven_victims',
       'Possible_victims', 'Notes', 'Nicknames', 'Age', 'Born_Date',
       'Born_Location', 'Died_Date', 'Died_Location', 'Conviction',
       'Criminal_Penalty', 'Date_Apprehended'],
      dtype='object')


In [88]:
def limpiar_nombres(nombre):
    if pd.isnull(nombre):
        return nombre
    nombre = nombre.strip()
    nombre = re.sub(r'["\'].*?["\']', '', nombre)  # quitamos apodos entre comillas
    nombre = re.sub(r'\(.*?\)', '', nombre)  # quitamos texto entre paréntesis
    nombre = " ".join(nombre.split())    # reemplazamos múltiples espacios por uno solo
    nombre = unicodedata.normalize("NFKD", nombre).encode("ASCII", "ignore").decode("utf-8")  # quitamos acentos
    return nombre.title()      # formato título


#aplicamos la funcion al dataframe
df_new_killers["Name"] = df_new_killers["Name"].apply(limpiar_nombres)


In [ ]:
#Una vez tenemos NAME que es el valor unico por el que nos guiamos normalizado pasamos a ver algunos datos de dtaframe que acabamos de crear
print("\nDuplicados por 'Name'")
duplicados = df_new_killers[df_new_killers.duplicated(subset='Name', keep=False)]
print(f"Total: {duplicados.shape[0]}")
if not duplicados.empty:
    display_cols = ['Name', 'Country', 'Start_year', 'End_year']
    print(duplicados[display_cols].sort_values(by='Name').head(10))

print("\nValores nulos por columna:")
print(df_new_killers.isnull().sum())

print("\nCasos donde End_year < Start_year")
errores_fecha = df_new_killers[df_new_killers['End_year'] < df_new_killers['Start_year']]
print(f"Total: {errores_fecha.shape[0]}")
if not errores_fecha.empty:
    print(errores_fecha[['Name', 'Start_year', 'End_year']].head(10))

print("\nRegistros con Proven_victims <= 0")
errores_victimas = df_new_killers[df_new_killers['Proven_victims'] <= 0]
print(f"Total: {errores_victimas.shape[0]}")
if not errores_victimas.empty:
    print(errores_victimas[['Name', 'Proven_victims']].head(10))

print("\nRegistros sin nombre:")
sin_nombre = df_new_killers['Name'].isnull().sum()
print(f"Total: {sin_nombre}")



Duplicados por 'Name'
Total: 141
                       Name         Country  Start_year  End_year
0                            United Kingdom      1968.0    1969.0
1                             United States      1972.0    1973.0
2                                Costa Rica      1986.0    1996.0
4                                    Brazil      2007.0    2008.0
5                                 Australia      1979.0    1983.0
7        Abboud And Khajawa             NaN         NaN       NaN
8        Abboud And Khajawa            Iraq         NaN    1917.0
9         Abdallah Al-Hubal             NaN         NaN       NaN
10        Abdallah Al-Hubal           Yemen      1990.0    1998.0
56  Angel Makers Of Nagyrev             NaN         NaN       NaN

Valores nulos por columna:
Name                  0
Country              95
Start_year           96
End_year            101
Proven_victims       95
Possible_victims     95
Notes                98
Nicknames           383
Age                 

In [93]:
#vamos a atajar esos duplicados para quedarnos con el que este más completo y eliminar el resto.
#vamos a crear una columna auxiliar que sera True si estan completos lo datos clave que marcamos
df_new_killers["completo"] = ~(
    df_new_killers["Country"].isnull() |
    df_new_killers["Start_year"].isnull() |
    df_new_killers["End_year"].isnull() |
    df_new_killers["Proven_victims"].isnull() |
    df_new_killers["Possible_victims"].isnull())

#ordenamos el df para que los registros con más datos sean los primeros
df_new_killers = df_new_killers.sort_values(by="completo", ascending=False)
#eliminamos duplicados por name quedandonos con la primera aparicion
df_new_killers = df_new_killers.drop_duplicates(subset="Name", keep="first")

df_new_killers.drop(columns="completo", inplace=True)

#comprobamos que ha funcionado
print("\nDuplicados por 'Name'")
duplicados = df_new_killers[df_new_killers.duplicated(subset='Name', keep=False)]
print(f"Total: {duplicados.shape[0]}")





Duplicados por 'Name'
Total: 0


### Pasamos a unir el dataframe inicial con el complementario

In [96]:
#Ver columnas disponibles en ambos
print(dfc.columns)
print(df_new_killers.columns)

#aplicamos la misma funcion de limpiar nombres que creamos antes
dfc["Name"] = dfc["Name"].apply(limpiar_nombres)

#Ver si hay coincidencias filtrando por la columna name
matches = dfc['Name'].isin(df_new_killers['Name']).sum()
print(f"Coincidencias por 'Name': {matches} de {len(dfc)}")

#Hacemos merge con outer para mantener todos los asesinos
df_final = pd.merge(dfc, df_new_killers, on='Name', how='outer', suffixes=('_kaggle', '_extra'))


Index(['Name', 'Country', 'Proven_victims', 'Possible_victims', 'Notes',
       'Start_year', 'End_year', 'Years_active'],
      dtype='object')
Index(['Name', 'Country', 'Start_year', 'End_year', 'Proven_victims',
       'Possible_victims', 'Notes', 'Nicknames', 'Age', 'Born_Date',
       'Born_Location', 'Died_Date', 'Died_Location', 'Conviction',
       'Criminal_Penalty', 'Date_Apprehended'],
      dtype='object')
Coincidencias por 'Name': 294 de 305


In [97]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 791 entries, 0 to 790
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Name                     791 non-null    object 
 1   Country_kaggle           305 non-null    object 
 2   Proven_victims_kaggle    305 non-null    float64
 3   Possible_victims_kaggle  237 non-null    object 
 4   Notes_kaggle             305 non-null    object 
 5   Start_year_kaggle        305 non-null    float64
 6   End_year_kaggle          305 non-null    float64
 7   Years_active             305 non-null    float64
 8   Country_extra            752 non-null    object 
 9   Start_year_extra         752 non-null    float64
 10  End_year_extra           746 non-null    float64
 11  Proven_victims_extra     752 non-null    float64
 12  Possible_victims_extra   752 non-null    float64
 13  Notes_extra              749 non-null    object 
 14  Nicknames                4

In [98]:
df_final.sample(10)

,Name,Country_kaggle,Proven_victims_kaggle,Possible_victims_kaggle,Notes_kaggle,Start_year_kaggle,End_year_kaggle,Years_active,Country_extra,Start_year_extra,End_year_extra,Proven_victims_extra,Possible_victims_extra,Notes_extra,Nicknames,Age,Born_Date,Born_Location,Died_Date,Died_Location,Conviction,Criminal_Penalty,Date_Apprehended
200,Eastbound Strangler,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,2006.0,2015.0,4.0,4.0,Killed four women in October and November 2006...,Atlantic City Serial Killer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
740,Viktor Sayenko And Igor Suprunyuk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ukraine,2007.0,2016.0,21.0,21.0,"""The Dnepropetrovsk Maniacs"". A pair of 19-yea...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
462,Marcelo Costa De Andrade,Brazil,14.0,NaN,Known as The Vampire of Niterói. Raped and kil...,1991.0,1991.0,1.0,Brazil,1991.0,2000.0,14.0,14.0,Known as The Vampire of Niterói. Raped and kil...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
604,Ripper Jayanandan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,India,2003.0,2006.0,7.0,7.0,Killed seven people during robberies. Sentence...,Ripper,NaN,NaN,"Thrissur, Kerala, India",NaN,NaN,Murder,deathby hanging,2006-11-23
781,Yoo Young-Chul,South Korea,21.0,26,"Used a hammer to murder mostly older victims, ...",2003.0,2004.0,2.0,South Korea,2003.0,2004.0,21.0,26.0,"Used a hammer to murder mostly older victims, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664,Sharon Kinne,United States,3.0,3,Sociopathic serial killer that murdered romant...,1960.0,1964.0,5.0,United States,1960.0,1964.0,3.0,3.0,Sociopathic serial killer that murdered romant...,"Jeanette Pugliese, La Pistolera",NaN,1939-11-30,NaN,NaN,NaN,Murder,NaN,NaN
393,Jurgen Bartsch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,West Germany,1962.0,1966.0,4.0,4.0,"Killed four, one escaped; died by wrongful ove...",The Carnival Killer (Der Kirmesmörder),29.0,1946-11-06,occupied Germany,1976-04-28,"Eickelborn, West Germany",Murder,"life imprisonment, commuted 10 years of juveni...",1966-01-01
537,Ottis Toole,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,1980.0,1983.0,8.0,112.0,"Initially convicted of three counts of murder,...",NaN,49.0,1947-03-05,NaN,1996-09-15,NaN,Murder Arson,Two death sentences plus four life sentence(s)...,1983-06-15
258,Gary C. Evans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,1985.0,1998.0,8.0,8.0,Befriended the Son of Sam killer while incarce...,NaN,43.0,1954-10-07,NaN,1998-08-14,NaN,NaN,None (died before trial),1998-05-27
391,Judy Buenoano,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,1971.0,1980.0,3.0,5.0,"Poisoned her husband, boyfriend and son with a...","Judias Goodyear, Judy Morris, Judias Buenoano,...",54.0,1943-04-04,NaN,1998-03-30,NaN,Murder,"life imprisonment (June 6, 1984)deathby electr...",1984-01-11


In [99]:
#Ya sabemos que Name es identificador único, ahora vamos a ver si hay duplicados por nombre.
duplicados = df_final[df_final.duplicated(subset='Name', keep=False)]

#En el caso de que haya, quiero ver los nombres que estan duplicados y cuantas veces se repite
nombres_duplicados = duplicados['Name'].value_counts()
print(f"Total asesinos con duplicados: {nombres_duplicados.shape[0]}")
print("Nombres duplicados (con cantidad de repeticiones):")
print(nombres_duplicados)


Total asesinos con duplicados: 0
Nombres duplicados (con cantidad de repeticiones):
Series([], Name: count, dtype: int64)


In [100]:
#Vemos las columnas comunes que tienen sufijos _kaggle y _extra
columnas_kaggle = [col for col in df_final.columns if col.endswith('_kaggle')]
columnas_extra = [col for col in df_final.columns if col.endswith('_extra')]

columnas_comunes = []
for col_k in columnas_kaggle:
    base_name = col_k.replace('_kaggle', '')
    col_e = base_name + '_extra'
    if col_e in columnas_extra:
        columnas_comunes.append(base_name)

#Analizar ambas direcciones
print("Aportes de datos entre _kaggle y _extra:")
for col in columnas_comunes:
    col_kaggle = f"{col}_kaggle"
    col_extra = f"{col}_extra"

    aporta_kaggle = df_final[col_extra].isnull() & df_final[col_kaggle].notnull()
    aporta_extra = df_final[col_kaggle].isnull() & df_final[col_extra].notnull()

    print(f" {col}")
    print(f"_kaggle aporta en: {aporta_kaggle.sum()} registros donde _extra está vacío")
    print(f"_extra aporta en: {aporta_extra.sum()} registros donde _kaggle está vacío")


Aportes de datos entre _kaggle y _extra:
 Country
_kaggle aporta en: 14 registros donde _extra está vacío
_extra aporta en: 461 registros donde _kaggle está vacío
 Proven_victims
_kaggle aporta en: 14 registros donde _extra está vacío
_extra aporta en: 461 registros donde _kaggle está vacío
 Possible_victims
_kaggle aporta en: 9 registros donde _extra está vacío
_extra aporta en: 524 registros donde _kaggle está vacío
 Notes
_kaggle aporta en: 14 registros donde _extra está vacío
_extra aporta en: 458 registros donde _kaggle está vacío
 Start_year
_kaggle aporta en: 14 registros donde _extra está vacío
_extra aporta en: 461 registros donde _kaggle está vacío
 End_year
_kaggle aporta en: 16 registros donde _extra está vacío
_extra aporta en: 457 registros donde _kaggle está vacío


In [101]:
#consolidamos las columnas rellenando los datos faltantes de una columna y otra y eliminamos
columnas_a_consolidar = ['Country', 'Proven_victims', 'Possible_victims', 'Notes', 'Start_year', 'End_year']

for col in columnas_a_consolidar:
    df_final[col] = df_final[f"{col}_extra"].combine_first(df_final[f"{col}_kaggle"])
    df_final.drop(columns=[f"{col}_kaggle", f"{col}_extra"], inplace=True)


In [102]:
#calculamos los registros faltates de Years_active 
df_final['Years_active'] = df_final['End_year'] - df_final['Start_year']
#y pasamos posibles victimas a numerico
df_final['Possible_victims'] = pd.to_numeric(df_final['Possible_victims'], errors='coerce')

In [103]:
#Como ya sabemos que Name es un id unico vamos a filtrar los asesinos de los que solo tenemos esa informacion para poder eliminarlos.
solo_nombre = df_final.drop(columns='Name').isnull().all(axis=1)
df_final = df_final[~solo_nombre].reset_index(drop=True)
print(f"Se eliminaron {solo_nombre.sum()} registros con solo el nombre.")



Se eliminaron 1 registros con solo el nombre.


In [104]:
#volvemos a ver los nulos por columnas
print("\nValores nulos por columna:")
print(df_new_killers.isnull().sum())


Valores nulos por columna:
Name                  0
Country              28
Start_year           28
End_year             34
Proven_victims       28
Possible_victims     28
Notes                31
Nicknames           341
Age                 558
Born_Date           374
Born_Location       397
Died_Date           498
Died_Location       569
Conviction          320
Criminal_Penalty    249
Date_Apprehended    287
dtype: int64


In [105]:
#La columna country es importante, vamos a pasar a limpiarla y a procesar los nulos.
#podemos usar Born_location para imputar los nulos de country
filtro = df_final['Country'].isnull() & df_final['Born_Location'].notnull()
#Sacamos el país que es la ultima parte de string
df_final.loc[filtro, 'Country'] = df_final.loc[filtro, 'Born_Location'].apply(lambda x: x.split(',')[-1].strip())
# Verificar cuántos se completaron
completados = filtro.sum()
print(f" País estimado desde Born_Location en {completados} registros.")
df_final['Country'].value_counts()
#DE nuevo encontramos paises que esta en una lista, repetimos el codigo que ejecutamos anteriormente para hacer registro unico por asesino y pais y luegos nos quedamos con la primera aparicion, las medas las aliminamos.
#Separamos los países multiples por coma y explotamos las filas
df_final['Country'] = df_final['Country'].str.split(',')
df_final = df_final.explode('Country')
#eliminamos espacios en blanco
df_final['Country'] = df_final['Country'].str.strip()
#dejamos solo una fila por asesino
df_final = df_final.drop_duplicates(subset='Name', keep='first').reset_index(drop=True)
#Pasamos a normalizar los paises, que hay mucho valores inconsistentes. Primero creamos un diccionario que le he pedido a chat gepete que me haga porque esto es perder la vida
normalizacion_paises = {
    # Variantes y abreviaturas
    "USA": "United States",
    "US": "United States",
    "United States of America": "United States",
    "United States of Brazil": "Brazil",
    "Rio de Janeiro": "Brazil",
    "So Paulo": "Brazil",
    "Mxico": "Mexico",
    "Uk": "United Kingdom",
    "UK": "United Kingdom",
    "England": "United Kingdom",
    "Scotland": "United Kingdom",
    "Ontario Canada": "Canada",
    "Territory of Hawaii": "United States",
    "British Hong Kong": "Hong Kong",
    "Imperial State of Iran": "Iran",

    # Históricos o alternativos
    "Allied-occupied Germany": "Germany",
    "West Germany": "Germany",
    "East Germany": "Germany",
    "Nazi Germany": "Germany",
    "German Empire, Germany": "Germany",
    "Soviet Union": "Russia",
    "Soviet Union, Russia": "Russia",
    "Soviet Union, Ukraine": "Ukraine",
    "Soviet Union, Russia, Ukraine": "Russia",
    "Uzbek SSR": "Uzbekistan",
    "UkSSR": "Ukraine",
    "Ottoman Empire": "Turkey",
    "centuryOttoman Empire": "Turkey",
    "Austria-Hungary": "Austria",
    "Austro-Hungarian Empire": "Austria",
    "Czechoslovakia": "Czech Republic",
    "SR Croatia": "Croatia",
    "SR Slovenia": "Slovenia",
    "Yugoslavia": "Serbia",
    "Kingdom of Yugoslavia": "Serbia",
    'Kingdom of Romania': 'Romania',
    "Kingdom of Romania, Yugoslavia, Hungary": "Romania",
    "Cape Colony": "South Africa",

    # Errores o indefinidos
    "TaeiIraq": "Iraq"
}
# Ejecutamos el diccionario 
df_final['Country'] = df_final['Country'].replace(normalizacion_paises)

print("Países únicos después de normalizar:")
print(sorted(df_final['Country'].dropna().astype(str).unique()))



 País estimado desde Born_Location en 16 registros.
Países únicos después de normalizar:
['Afghanistan', 'Argentina', 'Australia', 'Austria', 'Bangladesh', 'Belarus', 'Belgian Congo', 'Belgium', 'Belize', 'Brazil', 'Canada', 'Chile', 'China', 'Colombia', 'Costa Rica', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Ecuador', 'Egypt', 'Estonia', 'Finland', 'France', 'German Empire', 'Germany', 'Greece', 'Guatemala', 'Hong Kong', 'Hungary', 'India', 'Indonesia', 'Iran', 'Iraq', 'Israel', 'Italy', 'Japan', 'Kenya', 'Latvia', 'Malaysia', 'Mexico', 'Morocco', 'Namibia', 'Netherlands', 'Norway', 'Pakistan', 'Panama', 'Peru', 'Poland', 'Portugal', 'Republic of Macedonia', 'Romania', 'Russia', 'Rwanda', 'Serbia', 'Singapore', 'Slovenia', 'South Africa', 'South Korea', 'Spain', 'Sri Lanka', 'Swaziland', 'Sweden', 'Switzerland', 'Thailand', 'Tunisia', 'Turkey', 'Ukraine', 'United Kingdom', 'United States', 'Uzbekistan', 'Venezuela', 'Yemen', 'Zambia', 's Republic']


In [110]:
#continuamos filtrando los asesinos que tiene valores chungos en la columna país
# Filtrar asesinos con Country en los valores problemáticos
valores_chungos = ['s Republic']
df_chungos = df_final[df_final['Country'].isin(valores_chungos)]
display(df_chungos)

,Name,Years_active,Nicknames,Age,Born_Date,Born_Location,Died_Date,Died_Location,Conviction,Criminal_Penalty,Date_Apprehended,Country,Proven_victims,Possible_victims,Notes,Start_year,End_year
694,Tadeusz Grzesik,NaN,"Strawberry, Killer",NaN,NaN,s Republic,NaN,NaN,Murder,life imprisonment,2007-01-01,s Republic,NaN,NaN,NaN,NaN,NaN


In [111]:
#Imputamos los datos faltantes de este asesino con datos de wikipedia:
df_final.loc[df_final['Name'] == 'Tadeusz Grzesik', [
    'Nicknames', 'Conviction', 'Criminal_Penalty', 'Country',
    'Proven_victims', 'Possible_victims', 'Start_year', 'End_year',
    'Born_Date', 'Born_Location', 'Notes'
]] = [
    'Strawberry',                        # Nicknames
    'Murder',                            # Conviction
    'Life imprisonment',                 # Criminal_Penalty
    'Poland',                            # Country
    8,                                   # Proven_victims
    20,                                  # Possible_victims
    1991,                                # Start_year
    2007,                                # End_year
    '1960-01-01',                        # Born_Date
    'Wola Jachowa, Poland',              # Born_Location
    'A Polish strawberry grower and serial killer, known as "Strawberry" and "Killer". He led the Moneychangers Gang']

#verificamos
df_final[df_final['Name'] == 'Tadeusz Grzesik']





,Name,Years_active,Nicknames,Age,Born_Date,Born_Location,Died_Date,Died_Location,Conviction,Criminal_Penalty,Date_Apprehended,Country,Proven_victims,Possible_victims,Notes,Start_year,End_year
694,Tadeusz Grzesik,NaN,Strawberry,NaN,1960-01-01,"Wola Jachowa, Poland",NaN,NaN,Murder,Life imprisonment,2007-01-01,Poland,8.0,20.0,"A Polish strawberry grower and serial killer, ...",1991.0,2007.0


In [112]:
# Filtrar registros con Country nulo
nulos_country = df_final[df_final['Country'].isnull()]
display(nulos_country)



,Name,Years_active,Nicknames,Age,Born_Date,Born_Location,Died_Date,Died_Location,Conviction,Criminal_Penalty,Date_Apprehended,Country,Proven_victims,Possible_victims,Notes,Start_year,End_year
11,Adolfo Constanzo,NaN,The Godfather of Matamoros (El Padrino de Mata...,26.0,1962-11-01,NaN,1989-05-06,"Mexico City, Mexico",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,Charlie Chop-Off,NaN,"The Mad Barber, The Pied Piper of Manhattan",NaN,NaN,NaN,NaN,NaN,Never Convicted,Never sentenced,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,Dean Corll,NaN,"The Cy Man, The Pied Piper",33.0,1939-12-24,NaN,1973-08-08,NaN,Murder,Unattributable Due To Death,NaN,NaN,NaN,NaN,NaN,NaN,NaN
315,Ian Brady,NaN,The Moors Murderers,79.0,1938-01-02,NaN,2017-05-15,NaN,Murder,life imprisonment (whole life tariff),1965-10-07,NaN,NaN,NaN,NaN,NaN,NaN
333,Jason Thornburg,NaN,NaN,NaN,1980-08-27,NaN,NaN,NaN,Murder,death,NaN,NaN,NaN,NaN,NaN,NaN,NaN
351,John Allen Muhammad,NaN,"The Beltway Sniper, The D.C. Sniper",48.0,1960-12-31,NaN,2009-11-10,NaN,Capital Murder,death,2002-10-24,NaN,NaN,NaN,NaN,NaN,NaN
426,Leonard Lake,NaN,"Leonard Hill, Leonard Blake, Alan Drey, Tom Me...",39.0,1945-10-29,NaN,1985-06-06,NaN,Murder,NaN,1985-06-02,NaN,NaN,NaN,NaN,NaN,NaN
584,Raya And Sakina,NaN,NaN,NaN,NaN,NaN,NaN,Egypt,Murder,death,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Gestión de nulos

In [113]:
#Ver nulos por columna
total = len(df_final)
nulos_df = pd.DataFrame({
    'Nulos': df_final.isnull().sum(),
    'Porcentaje': (df_final.isnull().sum() / total * 100).round(2)}).sort_values(by='Nulos', ascending=False)
print("Nulos y porcentaje por columna:")
print(nulos_df)


Nulos y porcentaje por columna:
                  Nulos  Porcentaje
Died_Location       579       73.29
Age                 568       71.90
Died_Date           508       64.30
Born_Location       407       51.52
Born_Date           383       48.48
Nicknames           351       44.43
Conviction          330       41.77
Date_Apprehended    297       37.59
Criminal_Penalty    259       32.78
Possible_victims     32        4.05
Years_active         28        3.54
End_year             27        3.42
Notes                26        3.29
Proven_victims       23        2.91
Start_year           23        2.91
Country               8        1.01
Name                  0        0.00


In [114]:
#columnas como la ubicacion de nacimiento y muerte, tiene un alto porcentaje de nulos y no las vemos útiles para el analisis. 
#Decidimos borrarlas, pues ya las usamos para extraer datos interesantes como pais
df_final.drop(columns=['Died_Location', 'Born_Location'], inplace=True)

In [116]:
#Posible_victims y Proven_victims son esenciales para el analisis por lo que pasamos a borrar los registros que no tienen esos datos.
registros_sin_victimas = df_final['Proven_victims'].isnull() & df_final['Possible_victims'].isnull()
df_final = df_final[~registros_sin_victimas].reset_index(drop=True)
#comprobamos viendo de nuevo los nulos.
filtro = df_final['Proven_victims'].isnull() | df_final['Possible_victims'].isnull()
df_final.loc[filtro, ['Name', 'Proven_victims', 'Possible_victims']]


,Name,Proven_victims,Possible_victims
39,Anatoly Sedykh,12.0,NaN
50,Angelo Buono Jr.,10.0,NaN
202,El Psicopata,19.0,NaN
264,Giuseppe Greco,58.0,NaN
338,John Bunting,11.0,NaN
359,Jose Paz Bezerra,20.0,NaN
556,Rainbow Maniac,13.0,NaN
599,Robert Wagner,10.0,NaN
671,Surinder Koli,19.0,NaN


In [117]:
#Le paso nos nombre a chat gepete y le pido que busque informacion sobres las posibles victimas. 
#Tras la búsqueda decidimos que los asesinos que  no tenemos datos concisos se sustituya el nulo por 0 y en los casos que si tengamos datos se tomen esos datos para imputar los nulos.
df_final.loc[df_final['Name'] == 'José Paz Bezerra', ['Possible_victims']] = [24]
df_final.loc[df_final['Name'] == 'Rainbow Maniac', ['Possible_victims']] = [16]
df_final['Possible_victims'] = df_final['Possible_victims'].fillna(0)


In [118]:
#Volvemos a revisar nulos por columna
total = len(df_final)
nulos_df = pd.DataFrame({
    'Nulos': df_final.isnull().sum(),
    'Porcentaje': (df_final.isnull().sum() / total * 100).round(2)}).sort_values(by='Nulos', ascending=False)
print("Nulos y porcentaje por columna:")
print(nulos_df)

Nulos y porcentaje por columna:
                  Nulos  Porcentaje
Age                 552       71.97
Died_Date           494       64.41
Born_Date           372       48.50
Nicknames           338       44.07
Conviction          327       42.63
Date_Apprehended    286       37.29
Criminal_Penalty    254       33.12
Years_active          5        0.65
End_year              4        0.52
Notes                 3        0.39
Name                  0        0.00
Country               0        0.00
Proven_victims        0        0.00
Possible_victims      0        0.00
Start_year            0        0.00


In [119]:
#Queremos ver los asesinos que tiene vacio el registro de years_active
nulos_year = df_final[df_final['Years_active'].isnull()]
print(nulos_year.shape[0])
display(nulos_year)


5


,Name,Years_active,Nicknames,Age,Born_Date,Died_Date,Conviction,Criminal_Penalty,Date_Apprehended,Country,Proven_victims,Possible_victims,Notes,Start_year,End_year
19,Alejandro Maynez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mexico,2.0,50.0,"Killed at least two women in Ciudad Juárez, bu...",1980.0,NaN
74,Baba Anujka,NaN,"The Banat Witch, The Witch of Vladimirovac",NaN,NaN,NaN,Murder,15 years imprisonment with hard labour,1928-06-01,Serbia,50.0,150.0,Anujka was an accomplished amateur chemist fro...,1899.0,NaN
341,John Duffy And David Mulcahy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United Kingdom,3.0,100.0,"Known as the ""Railway Rapists"". Duffy and Mulc...",1982.0,NaN
494,Murder Incorporated,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,400.0,1000.0,Murder Incorporated or Murder Inc. for short w...,1920.0,NaN
672,Tadeusz Grzesik,NaN,Strawberry,NaN,1960-01-01,NaN,Murder,Life imprisonment,2007-01-01,Poland,8.0,20.0,"A Polish strawberry grower and serial killer, ...",1991.0,2007.0


In [120]:
#5 registros de End_year y start_year son asumibles. Pido a chat gepete que busque la info para esos asesinos, e imputar asi esos datos.
df_final.loc[df_final['Name'] == 'Baba Anujka', 'End_year'] = 1928
df_final.loc[df_final['Name'] == 'John Duffy and David Mulcahy', 'End_year'] = 1986
df_final.loc[df_final['Name'] == 'Murder Incorporated', 'End_year'] = 1940
df_final.loc[df_final['Name'] == 'Abboud and Khajawa', 'Start_year'] = 1917

In [121]:
#Combinando las columnas years_active, start_year y end_year podemos intentar completar los datos nulos de las mismas
# Rellenar Years_active si está vacío y hay Start y End
df_final['Years_active'] = df_final['Years_active'].fillna(df_final['End_year'] - df_final['Start_year'])

#Podemos seguir la misma estrategia con las columnas Age, Born_Date y Died_Date 
def calcular_edad(born, died):
    try:
        born = pd.to_datetime(born, errors='coerce')
        died = pd.to_datetime(died, errors='coerce')
        if pd.notnull(born) and pd.notnull(died):
            return died.year - born.year
        else:
            return None
    except:
        return None

df_final['Age_calculada'] = df_final.apply(lambda row: calcular_edad(row['Born_Date'], row['Died_Date']),axis=1)
df_final['Age'] = df_final['Age'].fillna(df_final['Age_calculada'])
df_final.drop(columns='Age_calculada', inplace=True)


In [122]:
#Volvemos a revisar nulos por columna
total = len(df_final)
nulos_df = pd.DataFrame({
    'Nulos': df_final.isnull().sum(),
    'Porcentaje': (df_final.isnull().sum() / total * 100).round(2)}).sort_values(by='Nulos', ascending=False)
print("Nulos y porcentaje por columna:")
print(nulos_df)

Nulos y porcentaje por columna:
                  Nulos  Porcentaje
Age                 552       71.97
Died_Date           494       64.41
Born_Date           372       48.50
Nicknames           338       44.07
Conviction          327       42.63
Date_Apprehended    286       37.29
Criminal_Penalty    254       33.12
Notes                 3        0.39
Years_active          2        0.26
End_year              2        0.26
Name                  0        0.00
Country               0        0.00
Proven_victims        0        0.00
Possible_victims      0        0.00
Start_year            0        0.00


In [123]:
# Eliminar columnas que tienen muchos nulos pues tras hablar con la PO nos anima a borrar los datos que no vamos a usar en nuestro análisis
df_final.drop(columns=['Age', 'Died_Date', 'Born_Date', 'Date_Apprehended'], inplace=True)

In [124]:
#reordenar columnas
column_order = ["Name", "Nicknames", "Country", "Proven_victims", "Possible_victims", "Years_active", "Start_year", "End_year", "Conviction", "Criminal_Penalty", "Notes"]
df_final = df_final[column_order]


Vemos que nuestro data set es un tanto escaso y decidimos completar con información haciendo un web scraping del sexo de los asesinos así como de las armas utilizadas.

In [125]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 767 entries, 0 to 766
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              767 non-null    object 
 1   Nicknames         429 non-null    object 
 2   Country           767 non-null    object 
 3   Proven_victims    767 non-null    float64
 4   Possible_victims  767 non-null    float64
 5   Years_active      765 non-null    float64
 6   Start_year        767 non-null    float64
 7   End_year          765 non-null    float64
 8   Conviction        440 non-null    object 
 9   Criminal_Penalty  513 non-null    object 
 10  Notes             764 non-null    object 
dtypes: float64(5), object(6)
memory usage: 66.0+ KB


In [126]:
df_scrapeo1 = pd.read_csv('Primary-data/scrapeo1.csv')

In [127]:
df_scrapeo1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 769 entries, 0 to 768
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    769 non-null    object
 1   Weapon  769 non-null    object
 2   Gender  769 non-null    object
dtypes: object(3)
memory usage: 18.2+ KB


In [130]:
#Vamos a unir los dos dataframes y aunque ya lo hicimos antes, normalizamos la columna name con nuestra super funcion 
# para estar bien seguras de que los datos son consistentes antes de hacer el merge.
df_final["Name"] = df_final["Name"].apply(limpiar_nombres)
df_scrapeo1["Name"] = df_scrapeo1["Name"].apply(limpiar_nombres)
#hacemos un left merge porque nos interesan mantener todos los asesinos de df_final que es el más completo.
df_completo = pd.merge(df_final, df_scrapeo1, on="Name", how="left")
#y revisamos nulos tras el merge
print(df_completo.isna().sum())




Name                  0
Nicknames           338
Country               0
Proven_victims        0
Possible_victims      0
Years_active          2
Start_year            0
End_year              2
Conviction          327
Criminal_Penalty    254
Notes                 3
Weapon                5
Gender                5
dtype: int64


In [131]:
# Filtra los registros donde Weapon o Gender sean nulos
nulos_weapon_gender = df_completo[df_completo["Weapon"].isna() | df_completo["Gender"].isna()]

# Muestra resultados
print(f"Registros con NaN en Weapon o Gender: {nulos_weapon_gender.shape[0]}")
display(nulos_weapon_gender)


Registros con NaN en Weapon o Gender: 5


,Name,Nicknames,Country,Proven_victims,Possible_victims,Years_active,Start_year,End_year,Conviction,Criminal_Penalty,Notes,Weapon,Gender
0,,NaN,United Kingdom,3.0,3.0,1.0,1968.0,1969.0,NaN,NaN,Unidentified serial killer who murdered three ...,NaN,NaN
347,"John Justin Bunting, Robert Joe Wagner And Jam...",NaN,Australia,12.0,12.0,7.0,1992.0,1999.0,NaN,NaN,The bodies of eight victims were found in plas...,NaN,NaN
390,Killer,NaN,Japan,12.0,12.0,9.0,1985.0,1994.0,NaN,NaN,Unidentified serial killer who carried out a s...,NaN,NaN
627,Satish,The Bahadurgarh baby killer,India,10.0,10.0,3.0,1995.0,1998.0,"Murder, Rape",Life sentence,Killed 10 girls who were between the ages of f...,NaN,NaN
647,Silvia Merez Cult,The Human Sacrifices Sect of Nacozari,Mexico,3.0,3.0,3.0,2009.0,2012.0,NaN,life imprisonment,Ritually murdered three people in sacrifices t...,NaN,NaN


In [132]:
#dedicimos eliminarlos por la ausencia de valores clave
df_completo = df_completo.dropna(subset=["Weapon", "Gender"], how="all")



In [133]:
#Vemos los valores unicos de weapon para poder establecer categorias para que sea más facil el analisis en tableau
valores_weapon = sorted(df_completo["Weapon"].dropna().unique())
for weapon in valores_weapon:
    print(weapon)


arsenic
arsenic, hanging
arsenic, injected
axe
axe, hammer, fire
axe, hanging
axe, hatchet, hanged
axe, strangled, strangulation, garrote
bat, strangled, hanged
beaten
beaten, strangled
beheaded, car
blunt object
blunt object, club, car, vehicle, truck
blunt object, club, fire, hanged
blunt object, poison, bomb, tortured, chemical
blunt object, strangled, burnt
blunt object, strangled, drowning, truck
blunt object, strangled, strangulation, car, vehicle, truck
burned
burned, hanged, hanging, car
burned, tortured
car
car, truck
car, vehicle
car, vehicle, chemical
car, vehicle, truck, tortured
chemical
choked
choked, drowned, car
choked, smothered, car
club
club, fire
club, poison, poisoned, arsenic, burned
club, strangled, burned, hanged
club, strangled, injected, car, tortured
club, strangulation, garrote, car, vehicle, tortured
club, strangulation, hanged, car
club, vehicle, truck, tortured
cyanide
decapitated
drowned
drowned, acid
drowning
drugs
explosives, car, tortured
fire
fire, a

In [134]:
#vamos a crear categorias para que así sea más legible el analisis posterior en tableau
def categorizar_weapon(valor):
    if pd.isna(valor):
        return "Unknown"
    valor = valor.lower()
    if any(p in valor for p in ["gun", "pistol", "revolver", "shotgun", "shooting", "shot"]):
        return "Firearm"
    elif any(p in valor for p in ["knife", "machete", "razor", "stabbed", "stabbing"]):
        return "Blade"
    elif any(p in valor for p in ["club", "bat", "hammer", "blunt object"]):
        return "Blunt Object"
    elif any(p in valor for p in ["strangled", "strangulation", "garrote", "choked", "smothered", "suffocated"]):
        return "Strangulation"
    elif any(p in valor for p in ["poison", "poisoned", "arsenic", "cyanide", "chemical", "chloroform"]):
        return "Poison"
    elif any(p in valor for p in ["car", "truck", "vehicle"]):
        return "Vehicle"
    elif any(p in valor for p in ["fire", "burned", "set on fire", "burnt", "explosives", "bomb", "grenade"]):
        return "Fire"
    elif any(p in valor for p in ["drowned", "drowning"]):
        return "Drowning"
    elif any(p in valor for p in ["beheaded", "decapitated"]):
        return "Beheading"
    elif any(p in valor for p in ["injected", "drugs", "medication"]):
        return "Drugs"
    elif any(p in valor for p in ["tortured", "beat to death", "incinerated", "disemboweled"]):
        return "Torture"
    elif any(p in valor for p in ["hanged", "hanging"]):
        return "Hanging"
    elif valor == "unknown":
        return "Unknown"
    else:
        return "Unknown" 


df_completo["Weapon_Category"] = df_completo["Weapon"].apply(categorizar_weapon)
print(df_completo["Weapon_Category"].value_counts())


Weapon_Category
Firearm          224
Blade            147
Strangulation    116
Unknown          101
Poison            47
Vehicle           38
Blunt Object      37
Hanging           23
Fire              11
Drowning           7
Drugs              5
Torture            5
Beheading          1
Name: count, dtype: int64


In [136]:
#Una vez tenemos el dataset limpio y completo de datos clave lo guardamos 
os.makedirs("Final-data", exist_ok=True)
output_path = "Final-data/final_final.csv"
df_completo.to_csv(output_path, index=False)
print(f"Archivo guardado correctamente en: {output_path}")

Archivo guardado correctamente en: Final-data/final_final.csv
